# Basic usage

*`skorch`* is designed to maximize interoperability between `sklearn` and `pytorch`. The aim is to keep 99% of the flexibility of `pytorch` while being able to leverage most features of `sklearn`. Below, we show the basic usage of `skorch` and how it can be combined with `sklearn`.

This notebook shows you how to use the basic functionality of `skorch`.

### Table of contents

* [Definition of the pytorch module](#Definition-of-the-pytorch-module)
* [Training a classifier](#Training-a-classifier-and-making-predictions)
  * [Dataset](#A-toy-binary-classification-task)
  * [pytorch module](#Definition-of-the-pytorch-classification-module)
  * [Model training](#Defining-and-training-the-neural-net-classifier)
  * [Inference](#Making-predictions,-classification)
* [Training a regressor](#Training-a-regressor)
  * [Dataset](#A-toy-regression-task)
  * [pytorch module](#Definition-of-the-pytorch-regression-module)
  * [Model training](#Defining-and-training-the-neural-net-regressor)
  * [Inference](#Making-predictions,-regression)
* [Saving and loading a model](#Saving-and-loading-a-model)
  * [Whole model](#Saving-the-whole-model)
  * [Only parameters](#Saving-only-the-model-parameters)
* [Usage with an sklearn Pipeline](#Usage-with-an-sklearn-Pipeline)
* [Callbacks](#Callbacks)
* [Grid search](#Usage-with-sklearn-GridSearchCV)
  * [Special prefixes](#Special-prefixes)
  * [Performing a grid search](#Performing-a-grid-search)

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
torch.manual_seed(0);

## Training a classifier and making predictions

### A toy binary classification task

We load a toy classification task from `sklearn`.

In [3]:
import numpy as np
from sklearn.datasets import make_classification

In [4]:
X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)

In [5]:
X.shape, y.shape, y.mean()

((1000, 20), (1000,), 0.5)

### Definition of the `pytorch` classification `module`

We define a vanilla neural network with two hidden layers. The output layer should have 2 output units since there are two classes. In addition, it should have a softmax nonlinearity, because later, when calling `predict_proba`, the output from the `forward` call will be used.

In [6]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
            dropout=0.5,
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin
        self.dropout = dropout

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(dropout)
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 2)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = F.relu(self.dense1(X))
        X = F.softmax(self.output(X))
        return X

### Defining and training the neural net classifier

We use `NeuralNetClassifier` because we're dealing with a classifcation task. The first argument should be the `pytorch module`. As additional arguments, we pass the number of epochs and the learning rate (`lr`), but those are optional.

*Note*: To use the cuda backend, pass `use_cuda=True` as an additional argument.

In [7]:
from skorch.net import NeuralNetClassifier

In [8]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    # use_cuda=True,  # uncomment this to train with CUDA
)

As in `sklearn`, we call `fit` passing the input data `X` and the targets `y`. By default, `NeuralNetClassifier` makes a `StratifiedKFold` split on the data (80/20) to track the validation loss. This is shown, as well as the train loss and the accuracy on the validation set.

In [9]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7111       0.5100        0.6894  0.0517
      2        0.6928       0.5500        0.6803  0.0395
      3        0.6833       0.5650        0.6741  0.0414
      4        0.6763       0.5850        0.6674  0.0415
      5        0.6727       0.6450        0.6616  0.0479
      6        0.6606       0.6600        0.6536  0.0482
      7        0.6560       0.6600        0.6443  0.0406
      8        0.6427       0.6650        0.6354  0.0465
      9        0.6300       0.6800        0.6264  0.0454
     10        0.6289       0.6800        0.6189  0.0479
     11        0.6241       0.7150        0.6114  0.0455
     12        0.6132       0.7150        0.6017  0.0509
     13        0.5950       0.7350        0.5902  0.0463
     14        0.5914       0.7200        0.5831  0.0428
     15        0.5784       0.7300        0.5733  0.0431
     16        0.5816       0.7

Also, as in `sklearn`, you may call `predict` or `predict_proba` on the fitted model.

### Making predictions, classification

In [10]:
y_pred = net.predict(X[:5])
y_pred

array([1, 0, 0, 0, 0])

In [11]:
y_proba = net.predict_proba(X[:5])
y_proba

array([[ 0.33409804,  0.66590196],
       [ 0.65906334,  0.34093666],
       [ 0.70409262,  0.29590738],
       [ 0.70345545,  0.29654452],
       [ 0.65079051,  0.34920952]], dtype=float32)

## Training a regressor

### A toy regression task

In [12]:
from sklearn.datasets import make_regression

In [13]:
X_regr, y_regr = make_regression(1000, 20, n_informative=10, random_state=0)
X_regr = X_regr.astype(np.float32)
y_regr = y_regr.astype(np.float32) / 100
y_regr = y_regr.reshape(-1, 1)

In [14]:
X_regr.shape, y_regr.shape, y_regr.min(), y_regr.max()

((1000, 20), (1000, 1), -6.4901485, 6.1545048)

*Note*: Regression currently requires the target to be 2-dimensional, hence the need to reshape. This should be fixed with an upcoming version of pytorch.

### Definition of the `pytorch` regression `module`

Again, define a vanilla neural network with two hidden layers. The main difference is that the output layer only has one unit and does not apply a softmax nonlinearity.

In [15]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=10,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

### Defining and training the neural net regressor

Training a regressor is almost the same as training a classifier. Mainly, we use `NeuralNetRegressor` instead of `NeuralNetClassifier` (this is the same terminology as in `sklearn`).

In [16]:
from skorch.net import NeuralNetRegressor

In [17]:
net_regr = NeuralNetRegressor(
    RegressorModule,
    max_epochs=20,
    lr=0.1,
    # use_cuda=True,  # uncomment this to train with CUDA
)

In [18]:
net_regr.fit(X_regr, y_regr)

  epoch    train_loss    valid_loss     dur
-------  ------------  ------------  ------
      1        4.6084        3.8757  0.0583
      2        3.9371        2.2167  0.0528
      3        1.0512        0.2644  0.0412
      4        0.2770        0.3933  0.0275
      5        0.3052        0.1762  0.0351
      6        0.1650        0.1601  0.0341
      7        0.1115        0.0990  0.0337
      8        0.1067        0.1418  0.0329
      9        0.0907        0.0828  0.0293
     10        0.0792        0.0760  0.0257
     11        0.0470        0.0529  0.0287
     12        0.0500        0.0426  0.0286
     13        0.0266        0.0365  0.0254
     14        0.0346        0.0255  0.0260
     15        0.0170        0.0269  0.0257
     16        0.0253        0.0170  0.0250
     17        0.0123        0.0213  0.0251
     18        0.0197        0.0122  0.0280
     19        0.0096        0.0177  0.0250
     20        0.0166        0.0104  0.0251


### Making predictions, regression

You may call `predict` or `predict_proba` on the fitted model. For regressions, both methods return the same value.

In [19]:
y_pred = net_regr.predict(X_regr[:5])
y_pred

array([[ 0.70368975],
       [-1.37799883],
       [-0.60438287],
       [ 0.0090515 ],
       [-0.52674961]], dtype=float32)

## Saving and loading a model

Save and load either the whole model by using pickle or just the learned model parameters by calling `save_params` and `load_params`.

### Saving the whole model

In [20]:
import pickle

In [21]:
file_name = '/tmp/mymodel.pkl'

In [22]:
with open(file_name, 'wb') as f:
    pickle.dump(net, f)

/home/marian/anaconda3/lib/python3.6/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type ClassifierModule. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [23]:
with open(file_name, 'rb') as f:
    new_net = pickle.load(f)

### Saving only the model parameters

This only saves and loads the proper `module` parameters, meaning that hyperparameters such as `lr` and `max_epochs` are not saved. Therefore, to load the model, we have to re-initialize it beforehand.

In [24]:
net.save_params(file_name)  # a file handler also works

In [25]:
# first initialize the model
new_net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
).initialize()

In [26]:
new_net.load_params(file_name)

## Usage with an `sklearn Pipeline`

It is possible to put the `NeuralNetClassifier` inside an `sklearn Pipeline`, as you would with any `sklearn` classifier.

In [27]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [28]:
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

In [29]:
pipe.fit(X, y)

Re-initializing module!
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6982       0.4950        0.6960  0.0654
      2        0.6943       0.5300        0.6943  0.0529
      3        0.6934       0.5100        0.6927  0.0442
      4        0.6924       0.5150        0.6912  0.0412
      5        0.6931       0.5100        0.6899  0.0422
      6        0.6897       0.5350        0.6891  0.0417
      7        0.6884       0.5450        0.6877  0.0394
      8        0.6821       0.5650        0.6856  0.0443
      9        0.6854       0.5800        0.6835  0.0394
     10        0.6801       0.6000        0.6813  0.0439
     11        0.6781       0.6150        0.6786  0.0455
     12        0.6758       0.6200        0.6757  0.0437
     13        0.6715       0.6250        0.6723  0.0432
     14        0.6711       0.6350        0.6686  0.0427
     15        0.6687       0.6350        0.6649  0.0467
     16

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('net', <skorch.net.NeuralNetClassifier object at 0x7fdbedce3828>)])

In [30]:
y_proba = pipe.predict_proba(X[:5])
y_proba

array([[ 0.52606189,  0.47393814],
       [ 0.56090653,  0.43909347],
       [ 0.58122021,  0.41877982],
       [ 0.58913922,  0.41086078],
       [ 0.58016819,  0.41983184]], dtype=float32)

To save the whole pipeline, including the pytorch module, use `pickle`.

## Callbacks

Adding a new callback to the model is straightforward. Below we show how to add a new callback that determines the area under the ROC (AUC) score.

In [31]:
from skorch.callbacks import Scoring
from skorch.utils import to_numpy

There is a scoring callback in skorch, `Scoring`, which we use for this. We need to specify a `name` of the score, as well as which score to calculate. Here we just pass a string, `'roc_auc_score'`, as score. For a list of all existing scores, look [here](http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics). We could also pass a function with the signature `func(model, X, y) -> score`, or `None`, in which case the `score` method of the model is used. Note that this is exactly the same behavior as in `sklearn`.

In [32]:
auc = Scoring(
    name='AUC',
    scoring='roc_auc_score',
    lower_is_better=False,
    pred_extractor=lambda y_proba: to_numpy(y_proba)[:, 1],
)

Furthermore, we should tell the callback that higher scores are better (to get the correct colors printed below), and how to extract the data from the prediction. The latter must be specified because sometimes we need the class predictions, sometimes the class probabilities, and sometimes, as in this example, only probability of the `1` class. Moreover, we must convert the data from `torch` tensors to `numpy` arrays (using the helper function `to_numpy`).

In [33]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    callbacks=[auc],
)

Finally, we pass the scoring callback to the `callbacks` parameter as a list and then call `fit`. Notice that we get the printed scores and color highlighting for free.

In [34]:
net.fit(X, y)

  epoch     AUC    train_loss    valid_acc    valid_loss     dur
-------  ------  ------------  -----------  ------------  ------
      1  0.5558        0.7150       0.4950        0.6935  0.0589
      2  0.6040        0.6811       0.5100        0.6756  0.0670
      3  0.6170        0.6694       0.5350        0.6661  0.0438
      4  0.6295        0.6505       0.5950        0.6557  0.0397
      5  0.6478        0.6447       0.6250        0.6457  0.0411
      6  0.6578        0.6389       0.6300        0.6359  0.0412
      7  0.6692        0.6282       0.6350        0.6277  0.0450
      8  0.6897        0.6159       0.6600        0.6172  0.0438
      9  0.7035        0.6085       0.6800        0.6096  0.0410
     10  0.7219        0.5993       0.6800        0.5996  0.0422
     11  0.7322        0.5950       0.6850        0.5941  0.0520
     12  0.7417        0.5813       0.6850        0.5869  0.0463
     13  0.7443        0.5853       0.6900        0.5815  0.0437
     14  0.7542        0.

For information on how to write custom callbacks, have a look at the *Advanced_Usage* notebook.

## Usage with sklearn `GridSearchCV`

### Special prefixes

The `NeuralNet` class allows to directly access parameters of the `pytorch module` by using the `module__` prefix. So e.g. if you defined the `module` to have a `num_units` parameter, you can set it via the `module__num_units` argument. This is exactly the same logic that allows to access estimator parameters in `sklearn Pipeline`s and `FeatureUnion`s.

This feature is useful in several ways. For one, it allows to set those parameters in the model definition. Furthermore, it allows you to set parameters in an `sklearn GridSearchCV` as shown below.

In addition to the parameters prefixed by `module__`, you may access a couple of other attributes, such as those of the optimizer by using the `optimizer__` prefix (again, see below). All those special prefixes are stored in the `prefixes_` attribute:

In [35]:
print(', '.join(net.prefixes_))

module, iterator_train, iterator_valid, optimizer, criterion, callbacks


### Performing a grid search

Below we show how to perform a grid search over the learning rate (`lr`), the module's number of hidden units (`module__num_units`), the module's dropout rate (`module__dropout`), and whether the SGD optimizer should use Nesterov momentum or not (`optimizer__nesterov`).

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    verbose=0,
    optimizer__momentum=0.9,
)

In [38]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20],
    'module__dropout': [0, 0.5],
    'optimizer__nesterov': [False, True],
}

In [39]:
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2)

In [40]:
gs.fit(X, y)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False, total=   0.7s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s


[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False, total=   0.7s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=False, total=   0.6s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True, total=   0.6s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True, total=   0.6s
[CV] lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True 
[CV]  lr=0.05, module__dropout=0, module__num_units=10, optimizer__nesterov=True, total=   0.7s
[CV] lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False 
[CV]  lr=0.05, module__dropout=0, module__num_units=20, optimizer__nesterov=False, total=   0.6s
[CV] lr=0.05, 

[CV]  lr=0.1, module__dropout=0.5, module__num_units=20, optimizer__nesterov=True, total=   0.7s


[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:   30.1s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=<skorch.net.NeuralNetClassifier object at 0x7fdc42416a58>,
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'lr': [0.05, 0.1], 'module__num_units': [10, 20], 'module__dropout': [0, 0.5], 'optimizer__nesterov': [False, True]},
       pre_dispatch='2*n_jobs', refit=False, return_train_score=True,
       scoring='accuracy', verbose=2)

In [41]:
print(gs.best_score_, gs.best_params_)

0.851 {'lr': 0.1, 'module__dropout': 0, 'module__num_units': 20, 'optimizer__nesterov': True}


Of course, we could further nest the `NeuralNetClassifier` within an `sklearn Pipeline`, in which case we just prefix the parameter by the name of the net (e.g. `net__module__num_units`).